## The data grabbed from UHF antenna

The Parallella (Pcko) grabbing unit was used to capture triggered data from loop antennas.

In [ ]:
# Tato bunka slouzi pro papermill, je oznacena tagem 'parameters'

path = "/storage/experiments/2022/0714_bourka_krivoklatsko_CAR2/parallella/"

session = [
    "/storage/experiments/2022/0714_bourka_krivoklatsko_CAR2/parallella/",
    "/storage/experiments/2022/0714_bourka_horovice_CAR1/parallella/",
    "/storage/experiments/2022/0714_bourka_CAR0/parallella/"
]

In [ ]:
# Coincidence detector
import pandas as pd
import numpy as np


#car0 = pd.read_csv("/storage/experiments/2022/0826_bourka_staryvestec_CAR0/triggers.csv", delimiter=";", names =["file", "datetime", "na"])
#car1 = pd.read_csv("/storage/experiments/2022/0826_bourka_staryvestec_CAR1/triggers.csv", delimiter=";", names =["file", "datetime", "na"])
#car2 = pd.read_csv("/storage/experiments/2022/0826_bourka_staryvestec_CAR2/triggers.csv", delimiter=";", names =["file", "datetime", "na"])

#car0 = pd.read_csv("/storage/experiments/2022/0827_bourka_morava_CAR0/triggers.csv", delimiter=";", names =["file", "datetime", "na"])
car1 = pd.read_csv("/storage/experiments/2022/0827_bourka_morava_CAR1/triggers.csv", delimiter=";", names =["file", "datetime", "na"])
car2 = pd.read_csv("/storage/experiments/2022/0827_bourka_CAR2/triggers.csv", delimiter=";", names =["file", "datetime", "na"])

car1["car"] = "CAR1"
car2["car"] = "CAR2"
#car0["car"] = "CAR0"

In [ ]:
cars = pd.concat([car0, car1, car2])
cars['ts'] = pd.to_datetime(cars['datetime'], utc=True)
cars = cars.sort_values(by=['ts']).reset_index()
cars['diff'] = cars['ts'].diff()
cars

In [ ]:
import datetime
clusters = []
opened = False

for i, r in enumerate(cars[::-1].iterrows()):
    j, d = r
    #print(i, d['diff'])
    if d['diff'] < datetime.timedelta(seconds=2) or opened:
        if not opened:
            clusters.append([])
            print(".......")
            opened = True
        if opened:
            clusters[-1].append(r)
            print(d['diff'].total_seconds(), d['car'], d['file'])
        if not d['diff'] < datetime.timedelta(seconds=2):
            opened = False
    else:
        opened = False
clusters

In [ ]:
for c in clusters:
    if len(c) > 2:
        print(c)

In [ ]:
import matplotlib 
matplotlib.use('Agg')

out = "/storage/experiments/2022/0826_bourka_staryvestec_CAR2/compare/"
for i, x in enumerate(clusters):
    try:
        plt.cla()
        plt.clf()
        if len(x) > 1:
            paths = []
            print("..........")
            for p in x:
                #print(p[1]['file']) 
                paths.append(p[1]['file'])
            print(">>", paths)
            multistation_plotrec(paths)
            print("Ukladam", out+str(i)+".png")
            filename = os.path.basename(p[1]['file'])
            plt.savefig(out+filename+".png")
            print("Ulozeno")
            #plt.xlim(14614528/20*2, 14614528/6)
            #plt.show()
    except Exception as e:
        print("chyba", e)

In [ ]:
import os
import sys
from readrec import *
import matplotlib
from scipy.signal import find_peaks


sps = 10e6 # 10 Msps


event = [
 "/storage/experiments/2022/0815_bourka_SlanyMelnik_CAR2/parallella/rec_220815-142820.68", 
 "/storage/experiments/2022/0815_bourka_kralupy_CAR0/parallella-VLF/rec_220815-142820.64",
# "/storage/experiments/2022/0815_bourka_benesov_CAR1/parallella/rec_220815-142821.68"
]

'''
event = [
    "/storage/experiments/2022/0815_bourka_SlanyMelnik_CAR2/parallella/rec_220815-144143.86",
    "/storage/experiments/2022/0815_bourka_kralupy_CAR0/parallella-VLF/rec_220815-144143.84"
]

event = [
   "/storage/experiments/2022/0815_bourka_SlanyMelnik_CAR2/parallella/rec_220815-150322.11",
    "/storage/experiments/2022/0815_bourka_kralupy_CAR0/parallella-VLF/rec_220815-150320.91"
]
'''

event = [
    "/storage/experiments/2022/0825_bourka_D8_9km_CAR0/parallella-VLF/rec_220825-171705.28",
    "/storage/experiments/2022/0825_bourka_D8_1km_CAR1/parallella/rec_220825-171706.27",
    "/storage/experiments/2022/0825_bourka_D8_19km_CAR2/parallella-VLF/rec_220825-171704.91"
]
out = "/storage/experiments/2022/0825_bourka_D8_19km_CAR2/Compare_220825-171704.png"

event = [
    "/storage/experiments/2022/0825_bourka_D8_9km_CAR0/parallella-VLF/rec_220825-171400.04",
    "/storage/experiments/2022/0825_bourka_D8_1km_CAR1/parallella/rec_220825-171359.92",
    "/storage/experiments/2022/0825_bourka_D8_19km_CAR2/parallella-VLF/rec_220825-171401.04"
]
out = "/storage/experiments/2022/0825_bourka_D8_19km_CAR2/Compare_220825-171401.png"

event = [
    "/storage/experiments/2022/0825_bourka_D8_9km_CAR0/parallella-VLF/rec_220825-171022.00",
    "/storage/experiments/2022/0825_bourka_D8_19km_CAR2/parallella-VLF/rec_220825-171023.92"
]
out = "/storage/experiments/2022/0825_bourka_D8_19km_CAR2/Compare_220825-171022.png"

event = [
    "/storage/experiments/2022/0825_bourka_D8_9km_CAR0/parallella-VLF/rec_220825-170347.25",
    "/storage/experiments/2022/0825_bourka_D8_1km_CAR1/parallella/rec_220825-170346.41",
    "/storage/experiments/2022/0825_bourka_D8_19km_CAR2/parallella-VLF/rec_220825-170348.45"
]
out = "/storage/experiments/2022/0825_bourka_D8_19km_CAR2/Compare_220825-170348.png"

# event = [
#     "/storage/experiments/2022/0825_bourka_D8_9km_CAR0/parallella-VLF/rec_220825-170158.35",
#     "/storage/experiments/2022/0825_bourka_D8_1km_CAR1/parallella/rec_220825-170158.57",
#     "/storage/experiments/2022/0825_bourka_D8_19km_CAR2/parallella-VLF/rec_220825-170200.51"
# ]
# out = "/storage/experiments/2022/0825_bourka_D8_19km_CAR2/Compare_220825-170200.png"


# event = [
# "/storage/experiments/2022/0825_bourka_D8_9km_CAR0/parallella-VLF/rec_220825-165949.66",
# "/storage/experiments/2022/0825_bourka_D8_1km_CAR1/parallella/rec_220825-165949.69",
# "/storage/experiments/2022/0825_bourka_D8_19km_CAR2/parallella-VLF/rec_220825-165950.96"
# ]

# out =  "/storage/experiments/2022/0825_bourka_D8_19km_CAR2/Compare_220825-165949.png"

# event = [
#     "/storage/experiments/2022/0825_bourka_D8_9km_CAR0/parallella-VLF/rec_220825-165814.09",
#     "/storage/experiments/2022/0825_bourka_D8_1km_CAR1/parallella/rec_220825-165814.43",
#     "/storage/experiments/2022/0825_bourka_D8_19km_CAR2/parallella-VLF/rec_220825-165814.34",
# ]

# out = "/storage/experiments/2022/0825_bourka_D8_19km_CAR2/Compare_220825-165814.png"

# event = [
#  "/storage/experiments/2022/0825_bourka_D8_9km_CAR0/parallella-VLF/rec_220825-164815.85",
#     "/storage/experiments/2022/0825_bourka_D8_1km_CAR1/parallella/rec_220825-164815.24",
#     "/storage/experiments/2022/0825_bourka_D8_19km_CAR2/parallella-VLF/rec_220825-164816.24"
# ]

# out = "/storage/experiments/2022/0825_bourka_D8_19km_CAR2/Compare_220825-164816.png"

event = ["/storage/experiments/2022/0826_bourka_staryvestec_CAR0/parallella-VLF/rec_220826-175301.51",
         "/storage/experiments/2022/0826_bourka_staryvestec_CAR1/parallella/rec_220826-175300.18"]
out = "/home/roman/test.png"

In [ ]:
def multistation_plotrec(files):
    
    channels = [0,1,3]
    fig, (ax) = plt.subplots(figsize=(35, 20), nrows=len(channels)*len(files), sharex=True)
    title = None
    
    data = {f:{"filename": f} for f in files}
    print(data)
    
    mintime = None
    
    for j, filepath in enumerate(data):
        print(j, filepath)
        try:
            with open(filepath, 'rb') as file:
                fn = os.path.basename(filepath)
                h, samples, synclog = loadrec(file)
                t2s, s2t, ticker, formatter = assign_time_axis(fn, h, synclog)

                time = s2t(0)
                if not mintime:
                    mintime = time
                if mintime > time:
                    mintime = time
        except Exception as e:
            print("Vyjimka 1", e)


    print("Znam minimalni cas", mintime)
    
    for j, filepath in enumerate(data):
        print(j, filepath)
        try:
            with open(filepath, 'rb') as file:
                fn = os.path.basename(filepath)

                h, samples, synclog = loadrec(file)
                t2s, s2t, ticker, formatter = assign_time_axis(fn, h, synclog)
                print("zacatek", s2t(0))
                offset = -int(t2s(mintime))
                #print("offset", offset)

                if title is not None:
                    fig.suptitle(title)

                for i, ch in enumerate(channels):
                    i += len(channels)*j
                    #print(i)

                    pre_trigger_blocks = h['preTrigger']
                    post_trigger_blocks = h['postTrigger']

                    a = (h['preTrigger']-pre_trigger_blocks)*h['descSpan']//16 #select block of samples before trigger
                    b = (h['preTrigger']+post_trigger_blocks)*h['descSpan']//16  #select blocks after trigger 
                    #print(a, b)

                    signal_samples = samples[a:b,ch]
                    #print(a, b, len(samples[:,0]))

                    axis_plotrec(ax[i], h, signal_samples, a+offset, b+offset, ticker, formatter, str("{}, channel: {}".format(filepath, ch)))
        except Exception as e:
            print("Vyjimka", e)
        print("Done", fn)

    return fig

In [ ]:
multistation_plotrec(event)
plt.savefig(out)
#plt.xlim(14614528/20*2, 14614528/6)
plt.show()

## Plotting of the data

In [ ]:
import os
pre_trigger_blocks = 380
post_trigger_blocks = 380


pre_trigger_blocks = 100
post_trigger_blocks = 100


matplotlib.rcParams.update({'font.size':15})

for fn in event:
    print(fn)
    if ".csv" in files:
        continue
    with open(os.path.join(path, fn), 'rb') as f:
        try:
            print("File:", fn)
            h, samples, synclog = loadrec(f)
            t2s, s2t, ticker, formatter = assign_time_axis(fn, h, synclog)
            fig = selective_plotrec(h, samples, synclog, os.path.basename(fn),
                    title=os.path.basename(fn),
                    pre_trigger_blocks = pre_trigger_blocks,
                    post_trigger_blocks = post_trigger_blocks,
                    marktimes=ticker,
                    channels = [0,1,3],
                    #channels = [0,1,2,3,4,5,6,7]
                    )
            
            plt.tight_layout()
            try:
                plt.savefig(os.path.join(path, fn)+'.png')
            except Exception as e:
                print(e)
            #plt.savefig('/home/roman/UHF.png')
            plt.show()
   
        except Exception as e:
            print(e)

Plot all lightning records (High memory and time consuming)

In [ ]:
from readrec import selective_plotrec